In [2]:
import numpy as np
import pandas as pd
import re
import requests
import json
import time


In [3]:
df = pd.read_csv('3. llm_outpu_new1.csv')
df.head()

,Index,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,...,Review Date,Product Description,Verified Purchase,Review Text,Helpful Votes,Cleaned Review Text,llm_raw,llm_parsed,Sentiment,Aspect-PainPoint Pairs
0,1,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Reviewed in India on 19 April 2025,Colour: Starlight|Size: 16GB Unified Memory|St...,True,I recently purchased the MacBook M4 with 512GB...,38,i recently purchased the macbook m4 with 512gb...,"Based on the review:\n\n{\n ""Sentiment"": ""Pos...","{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,NaN
1,2,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Reviewed in India on 4 April 2025,Colour: Midnight|Size: 16GB Unified Memory|Sty...,True,I placed the order on March 28th and received ...,118,i placed the order on march 28th and received ...,"Here is the output:\n\n{\n ""Sentiment"": ""Posi...","{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,NaN
2,3,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Reviewed in India on 9 May 2025,Colour: Sky Blue|Size: 16GB Unified Memory|Sty...,True,New to the apple ecosystem. Every single apple...,4,new to the apple ecosystem. every single apple...,The given review does not mention any specific...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,[]
3,4,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Reviewed in India on 4 July 2025,Colour: Starlight|Size: 16GB Unified Memory|St...,True,"Very easy to use , has met my expectations",1,"very easy to use , has met my expectations","Based on the review content, I extract the ins...","{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,NaN
4,5,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Reviewed in India on 19 April 2025,Colour: Starlight|Size: 16GB Unified Memory|St...,True,"Just got it 2 days ago. So far, just overwhelm...",25,"just got it 2 days ago. so far, just overwhelm...",Here is the output for the given review:\n\n{\...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,NaN


In [5]:
df.columns

Index(['Index', 'ASIN', 'Model Number', 'Manufacturer Name', 'Generic Name',
       'Product Title', 'Price', 'Overall Rating', 'Rating Count',
       'Review Count', 'Bought L Month', 'Product Link', 'Customer Say',
       'Insight', 'Individual Rating', 'Review Title', 'Review Date',
       'Product Description', 'Verified Purchase', 'Review Text',
       'Helpful Votes', 'Cleaned Review Text', 'llm_raw', 'llm_parsed',
       'Sentiment', 'Aspect-PainPoint Pairs'],
      dtype='object')

In [6]:
df.shape

(3842, 26)

In [8]:
df.isnull().sum()

Index                        0
ASIN                         0
Model Number                 0
Manufacturer Name            0
Generic Name                56
Product Title                0
Price                        0
Overall Rating               0
Rating Count                 0
Review Count                 0
Bought L Month            1226
Product Link                 0
Customer Say               555
Insight                    555
Individual Rating            1
Review Title                 0
Review Date                  0
Product Description        758
Verified Purchase            0
Review Text                  0
Helpful Votes                0
Cleaned Review Text          0
llm_raw                      1
llm_parsed                   0
Sentiment                   91
Aspect-PainPoint Pairs    1657
dtype: int64

In [10]:
df['Aspect-PainPoint Pairs'].unique()

array([nan, '[]', "['Battery - Short backup time']", ...,
       "['Performance - Average', 'Gaming - Not suitable']",
       "['Performance - Slow', 'Temperature - High heat', 'MS Office - Not functioning properly', 'Warranty - Poor after-sales service']",
       "['Battery - Poor backup time', 'Processor - Low performance']"],
      dtype=object)

In [12]:
df[df['Aspect-PainPoint Pairs']=="[]"]

,Index,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,...,Review Date,Product Description,Verified Purchase,Review Text,Helpful Votes,Cleaned Review Text,llm_raw,llm_parsed,Sentiment,Aspect-PainPoint Pairs
2,3,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Reviewed in India on 9 May 2025,Colour: Sky Blue|Size: 16GB Unified Memory|Sty...,True,New to the apple ecosystem. Every single apple...,4,new to the apple ecosystem. every single apple...,The given review does not mention any specific...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,[]
586,587,B0C4ZM63RP,Inspiron 3530,Dell,Laptop,"Dell Inspiron 3530 Thin & Light Laptop, 13th G...",33990,4.0,171,48,...,Reviewed in India on 13 April 2025,Colour: Carbon Black,True,Very good product,2,very good product,A new review to analyze!\n\nThe sentiment of t...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,[]
988,989,B0C28FKJXG,Lenovo V15 G2 Ijl,Lenovo,Laptop,"Lenovo V15 Intel Celeron N4500 15.6"" (39.62 cm...",21890,3.9,522,135,...,Reviewed in India on 14 November 2024,Pattern Name: Laptop,True,Good quality,1,good quality,A new review to analyze!\n\nThe sentiment of t...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,[]
1398,1399,B0DRCTY5JH,15IGL7,Lenovo,Laptop,Lenovo IdeaPad Slim 1 Intel® Celeron® N4020 15...,25500,3.6,320,74,...,Reviewed in India on 29 September 2024,"Size: 14"" | Intel Celeron",True,"Good laptop, value of money",0,"good laptop, value of money",A new review to analyze!\n\nThe sentiment is c...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,[]
1798,1799,B0D39BVJ7Y,lenovo v15 g4,Lenovo,Laptop,"Lenovo V15 AMD Ryzen 7 7730U 15.6"" (39.62cm) F...",42490,4.0,189,89,...,Reviewed in India on 5 June 2024,Size: Windows 11,True,Don't ever buy it,12,dont ever buy it,I can help with that!\n\nThe sentiment of this...,"{'Sentiment': 'Negative', 'Aspect-PainPoint Pa...",Negative,[]
2796,2797,B0CP2KRYLM,82R400ERIN,Lenovo,Laptop,"Lenovo IdeaPad Slim 1 AMD Ryzen 5 5500U 15.6"" ...",39990,4.0,360,89,...,Reviewed in India on 10 November 2024,Colour: Cloud Grey,True,Very good and comfortable,1,very good and comfortable,A new review to analyze!\n\nThe sentiment of t...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,[]
2945,2946,B0BSHVCXN9,MNWA3HN/A,Apple,"16-inch MacBook Pro (M2 Max, 2023)",Apple 2023 MacBook Pro Laptop with Apple M2 Ma...,329990,3.7,9,7,...,Reviewed in India on 19 December 2023,NaN,True,Excellent Product wonderful performance very p...,4,excellent product wonderful performance very p...,A new challenge!\n\nAfter analyzing the review...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,[]
2948,2949,B0BSHVCXN9,MNWA3HN/A,Apple,"16-inch MacBook Pro (M2 Max, 2023)",Apple 2023 MacBook Pro Laptop with Apple M2 Ma...,329990,3.7,9,7,...,Reviewed in India on 27 June 2024,NaN,False,"i gifted this tablet last week to my maid , an...",14,"i gifted this tablet last week to my maid , an...",What a lovely review!\n\nThe sentiment of this...,"{'Sentiment': 'Positive', 'Aspect-PainPoint Pa...",Positive,[]


In [14]:
df.loc[df['Aspect-PainPoint Pairs']=="[]", ['Aspect-PainPoint Pairs']] = None

In [16]:
df[df['Aspect-PainPoint Pairs']=="[]"]

,Index,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,...,Review Date,Product Description,Verified Purchase,Review Text,Helpful Votes,Cleaned Review Text,llm_raw,llm_parsed,Sentiment,Aspect-PainPoint Pairs


In [18]:
df.isnull().sum()

Index                        0
ASIN                         0
Model Number                 0
Manufacturer Name            0
Generic Name                56
Product Title                0
Price                        0
Overall Rating               0
Rating Count                 0
Review Count                 0
Bought L Month            1226
Product Link                 0
Customer Say               555
Insight                    555
Individual Rating            1
Review Title                 0
Review Date                  0
Product Description        758
Verified Purchase            0
Review Text                  0
Helpful Votes                0
Cleaned Review Text          0
llm_raw                      1
llm_parsed                   0
Sentiment                   91
Aspect-PainPoint Pairs    1665
dtype: int64

In [20]:
df.dropna(subset=['Aspect-PainPoint Pairs'], inplace=True)

In [22]:
df.shape

(2177, 26)

## 🧹 Standardizing Aspect-Pain Point Pairs (LLM Prompt)

This function constructs a structured prompt for the LLaMA 3 LLM to clean and standardize noisy `"Aspect - Pain Point"` pairs extracted from Amazon reviews.

### 🧠 Objective:
- Ensure consistent, concise, and meaningful phrasing for each aspect and its related pain point.
- Enforce a strict and clean format: `'Aspect - Pain Point'`

In [39]:
def build_pain_point_standardization_prompt(pain_points):
    return f"""
You are an AI assistant helping to clean and standardize 'Aspect - Pain Point' pairs extracted from Amazon product reviews.

Your goals:
- If the input is already clean and standardized, leave it as is.
- Clean and generalize the aspect if needed
- Clean and simplify the pain point
- Use concise and standardized wording
- Keep the format strictly as: 'Aspect - Pain Point'
- If the list is empty, return `null` (not a string)

### Formatting rules:
- Do NOT return any extra text — only the final cleaned JSON list
- Do NOT wrap the output in quotes (it should be a list, not a string)
- Do NOT output nested dictionaries or objects — only a flat list of strings in 'Aspect - Pain Point' format
- Preserve consistent casing (e.g., "Display", "Battery", "Customer Support")

### Examples:

Input:
['Display - Not great']  
Output:
['Display - Poor display quality']

Input:
['Audio - Low volume', 'Display - Poor viewing angles', 'Battery - Short backup time']  
Output:
['Audio - Low volume', 'Display - Poor viewing angles', 'Battery - Short backup time']

Input:
['Charging - Heating problem']  
Output:
['Charging - Heating issue']

Input:
['Storage (SSD) - Incorrect specification', 'Customer Service - Poor response to complaints']  
Output:
['Storage - Incorrect specification', 'Customer Support - Poor response']

Input:
['Battery - Short backup time']  
Output:
['Battery - Short backup time']

Input:
['Performance - Frequent freezing and system lag', 'Cooling - Extreme overheating', 'Noise - Loud fan noise', 'Network/Wi-Fi - Disconnects frequently', 'Customer Support - Unhelpful']

Output:
['Performance - Frequent freezing and system lag', 
 'Cooling - Extreme overheating', 
 'Noise - Loud fan noise', 
 'Network/Wi-Fi - Disconnects frequently', 
 'Customer Support - Unhelpful']

Now apply the same transformation logic to the following input:



- DO NOT add new aspects or pain points that are not present in the original list. Only clean or generalize what is already there.
- Retain any specific component names if mentioned. Do not generalize them away.
- Keep the number of items the same unless you're removing invalid entries (not adding).

Input:
{pain_points}

Return only a valid JSON list like:
[
  "Aspect - Standardized pain point",
  ...
]
"""


## 🧽 Cleaning & Standardizing Aspect–Pain Point Pairs with LLM

This block sends raw `"Aspect - Pain Point"` pairs to a local LLaMA 3 model and applies prompt-based standardization.

In [ ]:
def standardize_aspect_painpoints_llm(aspect_painpoints, idx=None):
    if pd.isna(aspect_painpoints):
        return None

    prompt = build_pain_point_standardization_prompt(aspect_painpoints)
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": "llama3", "prompt": prompt, "stream": False}
        )
        raw = response.json()["response"]
        match = re.search(r'\[.*\]', raw, re.DOTALL)
        if match:
            return json.loads(match.group())
        else:
            return aspect_painpoints  # fallback
    except Exception as e:
        print(f"❌ Error at row {idx}: {e}")
        return aspect_painpoints

standardized_aspect_painpoints = []

for idx, row in df.iterrows():
    raw_aspect_painpoints = row["Aspect-PainPoint Pairs"]
    cleaned = standardize_aspect_painpoints_llm(raw_aspect_painpoints, idx=idx)

    standardized_aspect_painpoints.append(cleaned)

    print(f"Row {idx}", end=" | ")
    print(f"🔹 Original: {raw_aspect_painpoints}", end=" | ")
    print(f"✅ Cleaned:  {cleaned}\n")

    time.sleep(1)

df["Standardized Aspect-PainPoint Pairs"] = standardized_aspect_painpoints

> ⚠️ Output of the above cell has been removed to keep the notebook readable. Below is sample output.

Row 5 | 🔹 Original: ['Battery - Short backup time'] | ✅ Cleaned:  ['Battery - Short backup time']

Row 11 | 🔹 Original: ['Processor - Low speed (0.01mhz)'] | ✅ Cleaned:  ['Processor - Slow performance']

Row 16 | 🔹 Original: ['Display - Not OLED/WUXGA quality', 'Battery - Short backup time (2-6 hours depending on usage)', 'Processor - Low gaming performance'] | ✅ Cleaned:  ['Display - Poor display quality', 'Battery - Limited battery life', 'Processor - Gaming performance issues']

Row 17 | 🔹 Original: [None] | ✅ Cleaned:  [None]

Row 18 | 🔹 Original: ['Performance - Frequent freezing and system lag', 'Cooling - Extreme overheating', 'Noise - Loud fan noise', 'Network/Wi-Fi - Disconnects frequently', 'Customer Support - Unhelpful'] | ✅ Cleaned:  ['Performance - Frequent system lag', 'Cooling - Extreme overheating', 'Noise - Loud fan noise', 'Network/Wi-Fi - Frequent disconnections', 'Customer Support - Unhelchar 1)  
Row 19 | 🔹 Original: ['Battery - Short backup time'] | ✅ Cleaned:  ['Battery - Short backup time']

Row 20 | 🔹 Original: ['Heat - Exhaust on bottom side'] | ✅ Cleaned:  ['Heat - Exhaust issue']


In [43]:
df.head(2)

,Index,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,...,Product Description,Verified Purchase,Review Text,Helpful Votes,Cleaned Review Text,llm_raw,llm_parsed,Sentiment,Aspect-PainPoint Pairs,Standardized Aspect-PainPoint Pairs
5,6,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Colour: Midnight|Size: 16GB Unified Memory|Sty...,True,Battery life is only 6 hours,35,battery life is only 6 hours,"Based on the given examples and rules, here's ...","{'Sentiment': 'Negative', 'Aspect-PainPoint Pa...",Negative,['Battery - Short backup time'],['Battery - Short backup time']
11,12,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,...,Colour: Starlight|Size: 16GB Unified Memory|St...,False,I am planning to but this model but I am seein...,10,i am planning to but this model but i am seein...,"Based on the review content, here's the output...","{'Sentiment': 'Neutral', 'Aspect-PainPoint Pai...",Neutral,['Processor - Low speed (0.01mhz)'],[Processor - Slow performance]


In [45]:
df.isnull().sum()

Index                                    0
ASIN                                     0
Model Number                             0
Manufacturer Name                        0
Generic Name                            29
Product Title                            0
Price                                    0
Overall Rating                           0
Rating Count                             0
Review Count                             0
Bought L Month                         711
Product Link                             0
Customer Say                           279
Insight                                279
Individual Rating                        1
Review Title                             0
Review Date                              0
Product Description                    393
Verified Purchase                        0
Review Text                              0
Helpful Votes                            0
Cleaned Review Text                      0
llm_raw                                  0
llm_parsed 

In [49]:
# df.to_csv('4. Final_Dataset.csv', index=False)